In [ ]:
pip install simpy

In [ ]:
import numpy as np
import sympy
import scipy.stats as spst
import scipy.special as spsp
import matplotlib.pyplot as plt
import simpy
import hyperopt
import copy

In [ ]:
def poisson_thinning_for_sys_sim(lm_max):
  N_arrivals=np.random.poisson(12*lm_max)
  arrivals=np.sort(np.random.rand(N_arrivals)*12)
  AR=(-0.01*(arrivals)**2+0.1*(arrivals)+1)/lm_max
  samples=arrivals[np.random.rand(N_arrivals)<AR]
  inter_arrival=np.diff(samples)
  inter_arrival=np.append(samples[0],inter_arrival)

  return inter_arrival

poisson_thinning_for_sys_sim(2)

array([0.34721428, 0.22567438, 0.61205355, 1.19846013, 0.6629473 ,
       2.36643474, 0.31182685, 0.18260544, 0.13372356, 1.27260237,
       0.57493253, 0.74548113, 1.22863423])

In [ ]:
np.append(10,np.array([1,2,3,4,5,6]))

array([10,  1,  2,  3,  4,  5,  6])

In [ ]:
x={"a":1,"b":2}
np.sum(list(x.values())

[1, 2]

In [ ]:
def system_sim(lmbda_M, lmbda_Q, lmbda_Bkn, lmbda_Brx, lmbda_S, vaccine_M, vaccine_Q, 
               vaccine_Bkn, vaccine_Brx, vaccine_S, M_serv, Q_serv, Bkn_serv, Brx_serv, S_serv, total_sim_time):
  env_M = simpy.Environment()
  env_Q = simpy.Environment()
  env_Bkn = simpy.Environment()
  env_Brx = simpy.Environment()
  env_S = simpy.Environment()
  desk_M = simpy.PriorityResource(env_M, capacity=400)
  desk_Q = simpy.PriorityResource(env_Q, capacity=400)
  desk_Bkn = simpy.PriorityResource(env_Bkn, capacity=400)
  desk_Brx = simpy.PriorityResource(env_Brx, capacity=200)
  desk_S = simpy.PriorityResource(env_S, capacity=100)

  Outputs = {"M":{"arrival":[],"start_vaccine":[],"end_vaccine":[],"vaccines_left":vaccine_M}, "Q":{"arrival":[],"start_vaccine":[],"end_vaccine":[],"vaccines_left":vaccine_Q}, 
             "Bkn":{"arrival":[],"start_vaccine":[],"end_vaccine":[],"vaccines_left":vaccine_Bkn}, "Brx":{"arrival":[],"start_vaccine":[],"end_vaccine":[],"vaccines_left":vaccine_Brx}, "S":{"arrival":[],"start_vaccine":[],"end_vaccine":[],"vaccines_left":vaccine_S}}
  
  Stop_sim_M = env_M.event()
  Stop_sim_Q = env_Q.event()
  Stop_sim_Bkn = env_Bkn.event()
  Stop_sim_Brx = env_Brx.event()
  Stop_sim_S = env_S.event()
  env_M.process(arrival(env_M, desk_M, lmbda_M, Outputs, Stop_sim_M, "M", total_sim_time, M_serv))
  env_Q.process(arrival(env_Q, desk_Q, lmbda_Q, Outputs, Stop_sim_Q, "Q", total_sim_time, Q_serv))
  env_Bkn.process(arrival(env_Bkn, desk_Bkn, lmbda_Bkn, Outputs, Stop_sim_Bkn, "Bkn", total_sim_time, Bkn_serv))
  env_Brx.process(arrival(env_Brx, desk_Brx, lmbda_Brx, Outputs, Stop_sim_Brx, "Brx", total_sim_time, Brx_serv))
  env_S.process(arrival(env_S, desk_S, lmbda_S, Outputs, Stop_sim_S, "S", total_sim_time, S_serv))
  env_M.run(until=Stop_sim_M)
  env_Q.run(until=Stop_sim_Q)
  env_Bkn.run(until=Stop_sim_Bkn)
  env_Brx.run(until=Stop_sim_Brx)
  env_S.run(until=Stop_sim_S)
  result_dict = {"Manhattan":Outputs["M"]["vaccines_left"], "Queens":Outputs["Q"]["vaccines_left"], 
                 "Brooklyn":Outputs["Bkn"]["vaccines_left"], "Bronx":Outputs["Brx"]["vaccines_left"], "Staten Island":Outputs["S"]["vaccines_left"]}
  a=list(result_dict.values())
  result=np.sum(a)
  return result

def poisson_thinning_for_sys_sim(lm_max, total_sim_time, borough_type):
  N_arrivals = np.random.poisson(total_sim_time*lm_max)
  arrivals = np.sort(np.random.rand(N_arrivals)*total_sim_time)
  if borough_type=="M":
    Acceptance_Rate = (-589.74725275*arrivals+17422.84615385)/lm_max
  elif borough_type=="Q":
    Acceptance_Rate = (-86.24175824*arrivals+117783.)/lm_max
  elif borough_type=="Bkn":
    Acceptance_Rate = (-272.70879121*arrivals+17691.57692308)/lm_max
  elif borough_type=="Brx":
    Acceptance_Rate = (-345.50549451*arrivals+9103.61538462)/lm_max
  else:
    Acceptance_Rate = (-158.46703297*arrivals+3450.11538462)/lm_max
  samples = arrivals[np.random.rand(N_arrivals)<Acceptance_Rate]
  inter_arrival = np.diff(samples)
  inter_arrival = np.append(samples[0],inter_arrival)

  return inter_arrival

def arrival(env, desk, lmbda, Outputs, Stop_sim, borough_type, total_sim_time, service_time):
  inter_arrivals = poisson_thinning_for_sys_sim(lmbda, total_sim_time, borough_type)
  
  for i in range(len(inter_arrivals)):
    inter_arrival = inter_arrivals[i]
    yield env.timeout(inter_arrival)
    Outputs[borough_type]["arrival"].append(env.now)
    if np.random.rand()<0.7:
      young_type = 1 # young people
    else:
      young_type = 0 # old people
    env.process(vaccination(env, desk, lmbda, Outputs, Stop_sim, borough_type, young_type, total_sim_time, service_time))
  # print(borough_type+": stop due to no more arrivals")
  Stop_sim.succeed()

def vaccination(env, desk, lmbda, Outputs, Stop_sim, borough_type, young_type, total_sim_time, service_time):
  # requesting the server
  if young_type==1:
    rqt = desk.request(priority=1, preempt=False) # less priority
  else:
    rqt = desk.request(priority=-1, preempt=False) # greater priority
  
  if Outputs[borough_type]["vaccines_left"]<=0 or env.now>=total_sim_time:
    # print(borough_type+": stop due to no more vaccines")
    # print(env.now)
    Stop_sim.succeed()

  # occupy/process the server request
  yield rqt
  Outputs[borough_type]["start_vaccine"].append(env.now)
  yield env.timeout(service_time)
  desk.release(rqt)
  Outputs[borough_type]["end_vaccine"].append(env.now)
  Outputs[borough_type]["vaccines_left"] -= 1


In [ ]:
system_sim(17422.84615385, 117783., 17691.57692308, 9103.61538462, 3450.11538462, 20000, 20000, 20000, 20000, 20000, 0.0034, 0.0034, 0.0034, 0.0034, 0.0034, 1)

33342

In [ ]:
def objective(params):
  vaccine_M = params[0]
  vaccine_Q = params[1]
  vaccine_Bkn = params[2]
  vaccine_Brx = params[3]
  vaccine_S = params[4]
  iteration_avg = np.mean([system_sim(17422.84615385, 117783., 17691.57692308, 9103.61538462, 3450.11538462, 
            vaccine_M, vaccine_Q, vaccine_Bkn, vaccine_Brx, vaccine_S, 5/(24*60), 5/(24*60), 5/(24*60), 5/(24*60), 5/(24*60), 1) for i in range(2)])
  return iteration_avg

trials=hyperopt.Trials()
best = hyperopt.fmin(fn=objective,
    space=[hyperopt.hp.quniform('vaccine_M', 5000, 20000, 500), hyperopt.hp.quniform('vaccine_Q', 5000, 20000, 500), hyperopt.hp.quniform('vaccine_Bkn', 5000, 20000, 500),
                        hyperopt.hp.quniform('vaccine_Brx', 5000, 20000, 500), hyperopt.hp.quniform('vaccine_S', 5000, 20000, 500)],
    ## random search
    #algo=hyperopt.rand.suggest,
    ##tpe search 
    algo=hyperopt.tpe.suggest,
    max_evals=20,
    trials=trials)

100%|██████████| 20/20 [02:02<00:00,  6.13s/it, best loss: 3031.5]


In [ ]:
print(best)

{'vaccine_Bkn': 14500.0, 'vaccine_Brx': 7000.0, 'vaccine_M': 18000.0, 'vaccine_Q': 7000.0, 'vaccine_S': 5500.0}


In [ ]:
system_sim(17422.84615385, 117783., 17691.57692308, 9103.61538462, 3450.11538462, 20000, 20000, 20000, 20000, 20000, 0.0034, 0.0034, 0.0034, 0.0034, 0.0034, 1)

MStop due to no more arrivals
QStop due to no more vaccines
0.17474128768086628
BknStop due to no more arrivals
BrxStop due to no more arrivals
SStop due to no more arrivals


{'Bronx': 11227,
 'Brooklyn': 2396,
 'Manhattan': 3106,
 'Queens': -2,
 'Staten Island': 16569}